In [2]:
import sys
sys.path.append("../")

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [9]:
import os 
import tensorflow as tf

from src.data.TextToToken.MyTfTokenizer import MyTfToken
from src.data.DataObjects.TransformerTextDataObject import TransformerTextDataObject
from src.data.DataLoaders import get_webscrape_data

from src.models.Transformer.Transformer import Transformer

from src.models.LossAndMetrics import masked_loss, masked_accuracy, CustomSchedule
from src.models.Callbacks.callbacks import csv_callback, checkpoint_callback, OutputTextCallback
from src.models.TextGenerators.StandardTransformerGenerator import StandardTransformerGenerator


#Project details
project_directory = os.path.abspath("../")
path_to_data_folder = os.path.join(project_directory, "data/processed/webdata")
context_token = MyTfToken(use_bookmark=True)
content_token = MyTfToken(use_bookmark=True)

model_name = "BuildingBert"

sequence_length = 62
batch_size = 64
buffer_size = 10000
embedding_dimension = 128
dense_dimension = 256
num_heads = 2
num_att_layers = 1
dropout_rate = 0.1


my_data_set = TransformerTextDataObject(context_sequencer=context_token, content_sequencer=content_token
                                        , context_len=sequence_length, content_len=sequence_length
                                        ,data_loader=get_webscrape_data, data_path=path_to_data_folder)

vocab_size_shake = my_data_set.content_vocab
vocab_size_eng = my_data_set.context_vocab

print(f"Shakespeare Vocab: {vocab_size_shake} , English Vocab: {vocab_size_eng}")

training_dataset = my_data_set.batch_and_shuffle(batch_size=batch_size,buffer_size=buffer_size)

Shakespeare Vocab: 11439 , English Vocab: 9114


In [10]:
import keras

In [11]:
bert_model_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"
bert_layer = hub.KerasLayer(bert_model_url, trainable=True)

In [12]:
model = keras.Sequential([
    Input(shape=(), dtype=tf.string),
    bert_layer,
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(vocab_size_shake, activation='softmax')
])

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x0000024C117D7340> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [31]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, AutoTokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [27]:
# Load pre-trained BERT model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
bert_model = TFBertModel.from_pretrained('google-bert/bert-base-uncased')

c:\Users\willi\Anaconda3\envs\ML_Env\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\willi\.cache\huggingface\hub\models--google-bert--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['

In [28]:
from keras import Sequential
from keras import Model

In [32]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFBertModel.from_pretrained("google-bert/bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [64]:
inputs = tokenizer("hello this is my brother", return_tensors="tf")
outputs = model(inputs)
pooled_output = outputs.pooler_output
x = Dense(256, activation='relu')(pooled_output)
output = Dense(vocab_size_shake, activation='softmax')(x)
predicted_label = tf.argmax(output, axis=-1).numpy()[0]
tokenizer.decode(predicted_label)

'suspected'

In [68]:
class BertModel(keras.Model):
    def __init__(self, vocab_size):
        super().__init__()
        self.bert_model = TFBertModel.from_pretrained("google-bert/bert-base-uncased")
        self.dense_comp = keras.layers.Dense(vocab_size, activation='softmax')
    def call(self, inputs):
        inputs = self.bert_model(inputs)
        pooled_output = inputs.pooler_output
        x = self.dense_comp(pooled_output)
        return x 

In [86]:
inputs = tokenizer("hello this is my brother prior hanging", return_tensors="tf")

In [87]:
A = BertModel(vocab_size_shake)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [88]:
B = A(inputs)
predicted_label = tf.argmax(B, axis=-1).numpy()[0]
tokenizer.decode(predicted_label)

'##ent'

'prior'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = TFBertModel.from_pretrained("google-bert/bert-base-uncased")

In [ ]:
trans_inst = Transformer(vocab_size=vocab_size_shake, context_vocab_size=vocab_size_eng
                         ,embedding_dimension=embedding_dimension, context_length=sequence_length
                         ,content_length=sequence_length, num_heads=num_heads
                         , dense_dimension=dense_dimension, num_att_layers=num_att_layers)